## Partition vs Coalesce

In [1]:
from pyspark.sql import (
    Row,
    SparkSession)
import pyspark.sql.functions as F

In [2]:
spark=(
    SparkSession
    .builder
    .appName("spark-partition")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 09:59:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

In [4]:
# partition 개수확인 
# getNumPartitions

In [5]:
# 파티션 개수 지정가능 
rdd1=sc.parallelize(range(20),1)
rdd4=sc.parallelize(range(20),4)

print(f"rdd1 파티션 개수: {rdd1.getNumPartitions()}") 
print(f"rdd4 파티션 개수: {rdd4.getNumPartitions()}") 

rdd1 파티션 개수: 1
rdd4 파티션 개수: 4


In [6]:
# 파티션 내부 확인 glom()

In [7]:
print("rdd1 contents:", rdd1.glom().collect())
print("rdd4 contents:", rdd4.glom().collect())

rdd1 contents: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]
rdd4 contents: [[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19]]


In [8]:
# 데이터 프레임도 파티션이다
# spark.range() 자동데이터 생성기 컬럼이름은 무조건 id고정 
# 파티션개수까지 정하기(start,end,step,partiton) 
# 0이상 20미만 1씩 증가하는 데이터를 5개 파티션에 담아줘
df=spark.range(0,20,1,5)
print("DF partitions:", df.rdd.getNumPartitions())
# df.show()

DF partitions: 5


In [9]:
# 파티션 내부가 너무 클때는 take사용
df.rdd.glom().take(2)

[[Row(id=0), Row(id=1), Row(id=2), Row(id=3)],
 [Row(id=4), Row(id=5), Row(id=6), Row(id=7)]]

In [10]:
# repartition vs coalesce 직접 비교

In [11]:
# repartiton 셔플 발생 

In [12]:
df_repart = df.repartition(8)
print("repartition:", df_repart.rdd.getNumPartitions())

repartition: 8


In [13]:
df_repart.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   ShuffleQueryStage 0
   +- Exchange RoundRobinPartitioning(8), REPARTITION_BY_NUM, [plan_id=15]
      +- *(1) Range (0, 20, step=1, splits=5)
+- == Initial Plan ==
   Exchange RoundRobinPartitioning(8), REPARTITION_BY_NUM, [plan_id=10]
   +- Range (0, 20, step=1, splits=5)




In [14]:
# coalesce (셔플 없음)

In [15]:
df_coal = df.coalesce(2)
print("coalesce:", df_coal.rdd.getNumPartitions())

coalesce: 2


In [16]:
df_coal.explain()

== Physical Plan ==
Coalesce 2
+- *(1) Range (0, 20, step=1, splits=5)




In [22]:
# 파티션 수가 성능에 미치는 영향 

In [23]:
import time

big_df = spark.range(0, 50_000_000)

def partition_duration(df,label):
    start=time.time()
    df.count()
    end=time.time()
    print(f"{label}: {end - start:.2f} seconds")

In [25]:
# 파티션 1개 
partition_duration(
    big_df.coalesce(1),
    "coalesce(1)"
)

coalesce(1): 0.71 seconds


In [28]:
# 파티션 8개로 늘리기
partition_duration(
    big_df.repartition(8),
    "repartition(8)"
)

[Stage 44:=======================================>                 (7 + 3) / 10]

repartition(8): 2.02 seconds


In [27]:
# 파티션 과다
partition_duration(
    big_df.repartition(200),
    "repartition(200)"
)

[Stage 38:======================>                                  (4 + 6) / 10]

repartition(200): 2.11 seconds


In [29]:
# coalesce(1): 0.71 seconds       <-- 가장 빠름! 
#repartition(8): 2.02 seconds    <--  느려짐 
#repartition(200): 2.11 seconds  <--  더 느려짐 

# 데이터가 아주 크지 않고 단순집계만 할때는 coalesce가 더 효율이 좋다는 것을 확인했다 
# repartition이 매번 좋은 건 아니다 오히려 단순한거에서 shuffle이 발생하여 오히려 느려졌다 
# 파티션이 과다해서 overhead가 추가되어 오히려 더 느려젔다